In [ ]:
import pandas as pd


# =======================================
# Class 1: Book Class
# Represents a book in the library.
# =======================================
class Book:
    def __init__(self, book_id, title, authors, average_rating, isbn, isbn13, language_code, num_pages, ratings_count,
                 text_reviews_count, publication_date, publisher, stock):  # Initialize Book Object
        self.book_id = book_id  # Unique identifier for the book
        self.title = title  # Title of the book
        self.authors = authors  # The name of the Author(s) of the book
        self.average_rating = average_rating  # Average rating of the book
        self.isbn = isbn  # ISBN of the book
        self.isbn13 = isbn13  # Language code of the book
        self.language_code = language_code  # Language code of the book
        self.num_pages = num_pages  # Number of pages in the book
        self.ratings_count = ratings_count  # Number of ratings for the book
        self.text_reviews_count = text_reviews_count  # Number of text reviews for the book
        self.publication_date = publication_date  # Publication date of the book
        self.publisher = publisher  # Publication date of the book
        self.stock = int(stock)  # Publication date of the book

    def __str__(self):  # Printing Book Object
        return (f"|{self.book_id} | {self.title} | {self.authors} | {self.publisher} "
                f"| {self.average_rating} | Stock: {self.stock}|")


# =======================================
# Class 2: User Class
# Represents a user in the library system
# =======================================
class User:
    def __init__(self, user_id, username, password, role="member"):  # Initialize User Object
        self.user_id = user_id  # Unique identifier for the user
        self.username = username  # Display name of the user
        self.password = password  # User's password for authentication
        self.borrowed_books = []  # List of books borrowed by the user
        self.role = role  # Role of the user, either 'admin' or 'member'

    def authenticate(self, password):  # Defines the authenticate function
        return self.password == password  # Compares the entered password with the stored password

    def borrow_book(self, book):  # Defines the function to borrow a book
        self.borrowed_books.append(book)  # Appends the book object to the user's borrowed book list

    def return_book(self, book_id):  # Defines the return_book function for the user

        for book in self.borrowed_books:  # Iterates through the list of borrowed books
            if book.book_id == book_id:  # Checks if the book ID matches the provided ID?
                self.borrowed_books.remove(book)  # Removes the book from the borrowed_books list
                return book  # Returns the book that was removed

        return None  # Returns None if the book is not found

    def __str__(self):  # Defines method to provide a readable string representation of the object
        # Creates a comma-separated string of borrowed book titles
        borrowed_titles = ', '.join(book.title for book in self.borrowed_books) or "No books borrowed"

        # Returns a formatted string with the user's ID, username, role, and borrowed books
        return f"|{self.user_id} | {self.username} | Role: {self.role} | Borrowed: {borrowed_titles}|"



# =========================================================================
# Class 3: LibraryManagementSystem Class
# Manages the library system, including books, users, and borrowing records
# =========================================================================
class LibraryManagementSystem:
    def __init__(self, books_csv, users_csv, borrow_csv):  # Initialize library object
        self.books_csv = books_csv  # Path to the books CSV file
        self.users_csv = users_csv  # Path to the users CSV file
        self.borrow_csv = borrow_csv  # Path to the borrowed books CSV file
        self.books = self.load_books()  # Load List of all books in the library
        self.users = self.load_users()  # List of all users in the library system
        self.merge_sort_users()  # Sorts the user by IDs using the merge sort algorithm
        self.load_borrowed_books()  # Loads borrowed book records
        self.last_sort_by = "book_id"  # Last sorting column for books

    def load_books(self):  # Load books from the books CSV file
        try:
            df = pd.read_csv(self.books_csv, dtype={"book_id": str, "title": str, "authors": str,
                                                    "isbn": str, "isbn13": str, "language_code": str,
                                                    "publication_date": str, "publisher": str})
            books = [Book(**row) for _, row in df.iterrows()]
            return books
            
        except FileNotFoundError: # File not exist
            print("Books CSV file not found. Starting with an empty library.")
            return []
            
        except Exception as other_err: # Other error
            print(f"An error occurred while loading books: {other_err}")
            return []

    def save_books(self): # Save books to the books CSV file.
        try:
            books_data = [vars(book) for book in self.books]
            df = pd.DataFrame(books_data)
            df.to_csv(self.books_csv, index=False)
        except Exception as other_err: # Other error
            print(f"An error occurred while saving books: {other_err}")

    def load_users(self): # Loads users from the users CSV file.
        try:
            df = pd.read_csv(self.users_csv, dtype={"user_id": str, "username": str, "password": str, "role": str})
            return [User(**row) for _, row in df.iterrows()]
            
        except FileNotFoundError: # File not exist
            print("Users CSV file not found. Starting with an empty user list.")
            return []
            
        except Exception as other_err: # Other error
            print(f"An error occurred while loading users: {other_err}")
            return []

    def save_users(self):  # Saves users to the users CSV file.
        try:
            users_data = [{'user_id': user.user_id, 'username': user.username, 
                       'password': user.password, 'role': user.role} for user in self.users]
            df = pd.DataFrame(users_data)
            df.to_csv(self.users_csv, index=False)
            
        except Exception as other_err: # Other error
            print(f"An error occurred while saving users: {other_err}")

    def load_borrowed_books(self): # Loads borrowed books from the borrow CSV file.
        try:
            df = pd.read_csv(self.borrow_csv, dtype={"user_id": str, "book_id": str})
            for _, row in df.iterrows():
                user = self.find_user(row['user_id'])
                book = next((b for b in self.books if b.book_id == row['book_id']), None)
                
                if user and book:
                    if book not in user.borrowed_books:  # Check if user and book exist?
                        user.borrow_book(book)
                    
                elif not user:
                    print(f"Warning: User ID {row['user_id']} not found.")
                
                elif not book:
                    print(f"Warning: Book ID {row['book_id']} not found.")
                    
        except FileNotFoundError: # File not exist
            print("Borrow CSV file not found. Starting with no borrowed books.")
            
        except Exception as other_err: # Other error
            print(f"An error occurred while loading borrowed books: {other_err}")

    def save_borrowed_books(self): # Saves borrowed books to the borrow CSV file.
        try:
            borrowed_data = []
            for user in self.users:
                for book in user.borrowed_books:
                    borrowed_data.append({'user_id': user.user_id, 'book_id': book.book_id})
                    
            if borrowed_data:   
                df = pd.DataFrame(borrowed_data)
            else:
                df = pd.DataFrame(columns=['user_id', 'book_id'])

            df.to_csv(self.borrow_csv, index=False)     
        except Exception as other_err: # Other error
            print(f"An error occurred while saving borrowed books: {other_err}")

    def view_books(self, limit=None):  # Displays available books in the library.
        print("\nAvailable Books:")
        print("|ID | Title | Author | Publisher | Rating| Stock|")
        
        if not self.books:
            print("No books available in the library.")
            return
             
        count = 0
        for book in self.books:
            print(book)
            count += 1

            if limit and count >= limit:  # Does it reach the limit?
                break

    def merge_sort_books(self, sort_by):  # Sorts the books using merge sort based on the specified column.
        if not hasattr(self.books[0], sort_by):  # Does the attribute exist?
            print(f"Invalid sort column '{sort_by}', no changes made.")  # Display invalid column
            return

        def merge_sort(arr):  # Helper function for merge sort
            if len(arr) > 1:  # Does the list have more than one element?
                mid = len(arr) // 2  # Calculate the middle index
                left_half = arr[:mid]  # Split the list into two halves
                right_half = arr[mid:]  # Split the list into two halves

                merge_sort(left_half)  # Recursively sort the left half
                merge_sort(right_half)  # Recursively sort the right half

                i = j = k = 0  # Initialize the counters
                while i < len(left_half) and j < len(right_half):

                    if getattr(left_half[i], sort_by).lower() \
                            if isinstance(getattr(left_half[i], sort_by), str) \
                            else (getattr(left_half[i], sort_by) < getattr(right_half[j], sort_by).lower()) \
                            if isinstance(getattr(right_half[j], sort_by), str) \
                            else getattr(right_half[j], sort_by):  # Does left value less than right half value?
                        arr[k] = left_half[i]
                        i += 1

                    else:
                        arr[k] = right_half[j]
                        j += 1

                    k += 1

                while i < len(left_half):  # Copy the left half
                    arr[k] = left_half[i]
                    i += 1
                    k += 1

                while j < len(right_half):  # Copy the right half
                    arr[k] = right_half[j]
                    j += 1
                    k += 1

        merge_sort(self.books)
        self.books = sorted(self.books, key=lambda x: getattr(x, sort_by))
        self.last_sort_by = sort_by
        print(f"Books sorted by '{sort_by}'.")

    def merge_sort_users(self):  # Sorts the users using merge sort by user ID.

        def merge_sort(arr):  # Helper function for merge sort
            if len(arr) > 1:  # Does the list have more than one element?
                mid = len(arr) // 2
                left_half = arr[:mid]
                right_half = arr[mid:]

                merge_sort(left_half)
                merge_sort(right_half)

                i = j = k = 0

                while i < len(left_half) and j < len(right_half):  # Merge the two halves
                    if left_half[i].user_id < right_half[j].user_id:  # Does left value less than right half value?
                        arr[k] = left_half[i]
                        i += 1
                    else:
                        arr[k] = right_half[j]
                        j += 1

                    k += 1

                while i < len(left_half):
                    arr[k] = left_half[i]
                    i += 1
                    k += 1

                while j < len(right_half):
                    arr[k] = right_half[j]
                    j += 1
                    k += 1

        merge_sort(self.users)

    def linear_search_books(self, keyword, search_by="title"):  # Search books with linear search on specified column.

        if not hasattr(self.books[0], search_by):  # Does the attribute exist?
            print(f"Invalid search column: '{search_by}'.")  # Display invalid column
            return  # Stop searching

        results = []

        for book in self.books:

            value = getattr(book, search_by)  # Get the book attribute

            if isinstance(value, str):  # Check the data type
                if search_by == "book_id":  # Book id must be exactly match
                    if keyword.lower() == value.lower():
                        results.append(book)
                elif keyword.lower() in value.lower():  # Other than book id check contains
                    results.append(book)

            elif isinstance(value, (int, float)):
                try:
                    numeric_keyword = float(keyword)
                    if value == numeric_keyword:  # Exact match for numeric column
                        results.append(book)
                except ValueError:
                    print(f"Invalid numeric value for search: {keyword}")
                    return

        if results:  # When we found a result
            print(f"\nSearch Results for: '{keyword}':")  # Display the keyword
            print("ID | Title | Author | Publisher | Rating| Stock|")  # Display the header

            for book in results:  # Display the result
                print(book)

        else:
            print(f"No books found for: '{keyword}'")  # Display No result

    def borrow_book(self, user_id, book_id):  # Allows a user to borrow a book.
        user = self.find_user(user_id)  # Find the user by ID

        if user is None:  # Check if user not found
            print("\nUser ID not found.")  # Display user not found
            return  # Stop the process

        book = next((b for b in self.books if b.book_id == book_id), None)  # Find the book by ID

        if book:  # Check if book found
            user.borrow_book(book)  # Borrow the book
            print(f"\n'{user.username}' borrowed: '{book.title}'.")
            self.save_borrowed_books()
            
        else:  # Book not exist or already borrowed before
            print("\nBook ID not found or it is already borrowed.")

    def return_book(self, user_id, book_id):  # Allows a user to return a book
        user = self.find_user(user_id)  # Finding user

        if user is None:  # Does user exist?
            print("\nUser ID not found.")
            return  # Stop the process

        book = user.return_book(book_id)  # Returning the book

        if book:  # Do we have the book?
            print(f"\n'{user.username}' returned: '{book.title}'.")
        else:  # Book not exist
            print("\nBook not found in user's borrowed books.")

        return book  # Return None or the book data

    def remove_book(self, book_id):  # Removing a book from the library

        for book in self.books:

            if book.book_id == book_id:  # Does the book id match?
                self.books.remove(book)  # Remove the book from the library
                self.save_books() # save remove book to books.csv
                print(f"\nBook '{book.title}' removed successfully")
                return  # Remove process finish

        print("\nBook ID not found")

    def add_book(self, book_data):  # Allows a user to add a book.
        try:
            # Check if book_id already exists 
            if any(book.book_id == book_data['book_id'] for book in self.books):
                print(f"Book ID '{book_data['book_id']}' already exists in the library. Cannot add duplicate.")
                return  # Process aborted due to book_id exists
            
            required_fields = ['title', 'authors', 'average_rating', 'isbn', 'isbn13',
                               'language_code', 'num_pages', 'ratings_count', 'text_reviews_count',
                               'publication_date', 'publisher', 'stock']  # Fields to be required

            for field in required_fields:  # Checking all required fields
                if field not in book_data or not book_data[field].strip():  # Do any fields empty?
                    print(f"Missing or invalid data for field: {field}")
                    return  # Process aborted due to invalid data

            new_book = Book(
                book_data['book_id'], book_data['title'], book_data['authors'], book_data['average_rating'],
                book_data['isbn'], book_data['isbn13'], book_data['language_code'], book_data['num_pages'],
                book_data['ratings_count'], book_data['text_reviews_count'], book_data['publication_date'],
                book_data['publisher'], book_data['stock'])  # Create book object

            self.books.append(new_book)  # Add the book to the library
            self.merge_sort_books(self.last_sort_by)  # Sort the book
            self.save_books()  # save add book to books.csv
            print(f"\nBook '{book_data['title']}' added to the library and sorted by '{self.last_sort_by}'.")

        except Exception as other_err:  # Other error
            print(f"An error occurred: {other_err}")

    def update_book(self, book_id, updated_data):  # Allows a user to update a book.
        try:
            for book in self.books:
                if book.book_id == book_id:  # Does the book ID match?
                    for key, value in updated_data.items():
                        if hasattr(book, key):  # Does attribute exist?
                            setattr(book, key, value)  # Set new value to the object

                    self.merge_sort_books(self.last_sort_by)  # Sort the book
                    self.save_books() # save upadte book to books.csv
                    print(f"\nBook '{book.title}' updated and sorted by '{self.last_sort_by}'.")
                    return  # Update process finish

            print("\nBook ID not found.")
        except Exception as other_err:  # Other error
            print(f"An error occurred: {other_err}")

    def add_user(self, user_id, username, password, role):  # Adds a new user to the library system.
        if any(user.user_id == user_id for user in self.users):  # Does the user exist?
            print("\nUser ID already existed.")
            return

        new_user = User(user_id, username, password, role)  # Create user object
        self.users.append(new_user)
        self.merge_sort_users()
        self.save_users()
        print(f"\nUser '{username}' added successfully.")

    def delete_user(self, user_id):  # Deletes a user from the library system.
        for user in self.users:
            if user.user_id == user_id:  # Does the user id match?
                self.users.remove(user)
                print(f"\nUser '{user.username}' deleted successfully.")
                self.save_users() 
                return

        print("\nUser ID not found.")

    def update_user(self, user_id, updated_data):  # Updates user information.
        user = self.find_user(user_id)

        if not user:
            print("\nUser ID not found.")
            return

        for key, value in updated_data.items():
            if hasattr(user, key):
                setattr(user, key, value)
        self.save_users()

        print(f"\nUser '{user.username}' updated successfully.")

    def find_user(self, user_id):  # Finds a user by ID using binary search.
        left, right = 0, len(self.users) - 1

        while left <= right:  # Binary search
            mid = (left + right) // 2  # Calculate the middle index

            if self.users[mid].user_id == user_id:  # Check if the user ID matches
                return self.users[mid]  # Return the user if found
            elif self.users[mid].user_id < user_id:  # user_id < mid
                left = mid + 1
            else:  # user_id > mid
                right = mid - 1

        return None  # The user if found, None otherwise.

    def view_users(self):  # Displays all available users in the library.
        print("\nRegistered Users:")
        print("ID | Username | Borrowed Books")

        for user in self.users:
            print(user)

    def admin_menu(self):  # Displays the menu for admin users and handles their actions.
        while True:  # Loop for admin menu
            print("\nAdmin Menu")
            print("1. Add User")
            print("2. View Users")
            print("3. Update User")
            print("4. Delete User")
            print("5. Add Book")
            print("6. Remove Book")
            print("7. Update Book")
            print("8. Logout")
            choice = input("Enter your choice: ")

            if choice == '1':  # Add User
                user_id = input("Enter User ID: ").strip()
                username = input("Enter Username: ").strip()
                password = input("Enter Password: ").strip()
                role = input("Enter Role [admin/member]: ").strip()
                self.add_user(user_id, username, password, role)

            elif choice == '2':  # View Users
                self.view_users()

            elif choice == '3':  # Update User
                user_id = input("Enter User ID to update: ").strip()
                updated_data = {}

                while True:
                    key = input("Enter the field to update e.g username, password [or 'done' to finish]: ").strip()

                    if key.lower() == 'done':
                        break
                    if key not in ['username', 'password', 'role']:
                        print("Invalid field, try again!")
                        continue

                    value = input(f"Enter the new value for '{key}': ").strip()
                    updated_data[key] = value

                self.update_user(user_id, updated_data)

            elif choice == '4':
                user_id = input("Enter User ID to delete: ").strip()
                self.delete_user(user_id)

            elif choice == '5':
                book_data = {
                    'book_id': input("Enter Book ID: "),
                    'title': input("Enter Title: "),
                    'authors': input("Enter Author: "),
                    'average_rating': input("Enter Average Rating: "),
                    'isbn': input("Enter ISBN: "),
                    'isbn13': input("Enter ISBN13: "),
                    'language_code': input("Enter Language Code: "),
                    'num_pages': input("Enter Number of Pages: "),
                    'ratings_count': input("Enter Ratings Count: "),
                    'text_reviews_count': input("Enter Text Reviews Count: "),
                    'publication_date': input("Enter Publication Date: "),
                    'publisher': input("Enter Publisher: "),
                    'stock': input("Enter Stock: ")
                }
                self.add_book(book_data)

            elif choice == '6':
                book_id = input("Enter Book ID to remove: ").strip()
                self.remove_book(book_id)

            elif choice == '7':
                book_id = input("Enter the Book ID to update: ").strip()
                updated_data = {}

                while True:
                    key = input("Enter the field to update [or 'done' to finish]: ").strip()

                    if key.lower() == 'done':
                        break

                    if key not in ['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages',
                                   'ratings_count', 'text_reviews_count', 'publication_date', 'publisher', 'stock']:
                        print("Invalid field, try again!")
                        continue

                    value = input(f"Enter the new value for '{key}': ").strip()
                    updated_data[key] = value

                self.update_book(book_id, updated_data)

            elif choice == '8':
                print("Logging out...")
                break

            else:  # Invalid choice
                print("Invalid choice, please try again!")

    def member_menu(self, member):  # Displays the menu for member users and handles their actions.
        while True:
            print("\nMember Menu")
            print("1. View Books")
            print("2. Search Books")
            print("3. Borrow Book")
            print("4. Return Book")
            print("5. Sort Books")
            print("6. Check Profile")
            print("7. Update Profile")
            print("8. Delete Account")
            print("9. Logout")
            choice = input("Enter your choice: ")

            if choice == '1':
                limit = input("Enter the number of books to view [leave blank to view all]: ").strip()
                limit = int(limit) if limit.isdigit() else None
                self.view_books(limit)

            elif choice == '2':
                search_by = input("Enter books  column to search e.g by title, book_id, authors, etc [default: title]: ").strip() or "title"
                keyword = input("Enter a keyword to search: ")
                self.linear_search_books(keyword, search_by)

            elif choice == '3':
                book_id = input("Enter Book ID to borrow: ").strip()
                book = next((b for b in self.books if b.book_id == book_id), None)

                if book and book.stock > 0:
                    self.borrow_book(member.user_id, book_id)
                    book.stock -= 1
                    self.save_books()
                    self.save_borrowed_books()
                else:
                    print("Book is not available for borrowing.")

            elif choice == '4':
                book_id = input("Enter Book ID to return: ").strip()
                book = self.return_book(member.user_id, book_id)

                if book:
                    book.stock += 1
                    self.save_books()
                    self.save_borrowed_books()

            elif choice == '5':
                sort_by = input("Enter the column to sort books by [title, book_id, authors, stock, etc]: ").strip()
                self.merge_sort_books(sort_by)
                self.view_books()

            elif choice == '6':
                print(member)

            elif choice == '7':
                updated_data = {}

                while True:  # Update Profile
                    key = input("Enter the field to update [or 'done' to finish]: ").strip()
                    if key.lower() == 'done':
                        break

                    if key not in ['username', 'password']:
                        print("Invalid field, try again!")
                        continue

                    value = input(f"Enter the new value for '{key}': ").strip()
                    updated_data[key] = value

                self.update_user(member.user_id, updated_data)

            elif choice == '8':
                confirm = input("Are you sure you want to delete your account? [yes/no]: ").strip().lower()

                if confirm == 'yes':
                    self.delete_user(member.user_id)
                    break

            elif choice == '9':
                print("Logging out...")
                break

            else:
                print("Invalid choice, please try again!")

    def run(self):  # Main entry point for the library management system.
        while True:
            print("\nWelcome to the Library Management System")
            print("1. Register as a Member")
            print("2. Login as Admin")
            print("3. Login as Member")
            print("4. Exit")
            choice = input("Enter your choice: ")

            if choice == '1':
                user_id = input("Enter User ID: ").strip()
                username = input("Enter Username: ").strip()
                password = input("Enter Password: ").strip()
                role = 'member'
                self.add_user(user_id, username, password, role)

            elif choice == '2':
                user_id = input("Enter Admin ID - see users.csv or use U001: ").strip()
                password = input("Enter Password - see users.csv or use 123 : ").strip()
                user = self.find_user(user_id)

                if user and user.authenticate(password) and user.role == 'admin':
                    self.admin_menu()
                else:
                    print("Invalid Admin ID or Password!")

            elif choice == '3':
                user_id = input("Enter Member ID - see users.csv or use U002: ").strip()
                password = input("Enter Password - see users.csv or use 456: ").strip()
                user = self.find_user(user_id)

                if user and user.authenticate(password) and user.role == 'member':
                    self.member_menu(user)
                else:
                    print("Invalid Member ID or Password!")

            elif choice == '4':
                print("Exiting the system. Goodbye!")
                break

            else:
                print("Invalid choice, please try again!")

# =======================================
# Main Program: 
# =======================================
if __name__ == "__main__":  # Run the program
    library = LibraryManagementSystem('books.csv', 'users.csv', 'borrow.csv')
    library.run()



Welcome to the Library Management System
1. Register as a Member
2. Login as Admin
3. Login as Member
4. Exit
